In [1]:
!pip install jsonapi_client

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# declarations:
biom = GALAXY_INPUTS['biom']
max_sample_count = GALAXY_INPUTS['count']

# rework max_sample_count.




In [3]:
from jsonapi_client import Session
import pandas as pd
import numpy as np
import requests
import os

###################
# Definie Functions
###################


sample_accessions = []



# retuns list of id`s for all Studies to Biom
def GetBiomSamplesByIds(biom_id: str):
    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        i = 0
        biomes_dfs = []
        for r in mgnify.iterate(f'biomes/{biom_id}/samples'):
            biom_df = pd.json_normalize(r.json)
            biom_df['url'] = str(r.links.self)
            biomes_dfs.append(biom_df)
            i += 1
            if(i == max_sample_count):
                break
        
    main_biomes_df = pd.concat(biomes_dfs)

    # main_biomes_df = main_biomes_df.dropna()

    return main_biomes_df

def GetAnalysisFromStudie(stydy_id):
    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        i = 0
        biomes_dfs = []
        for r in mgnify.iterate(f'studies/{stydy_id}/analyses'):
            biom_df = pd.json_normalize(r.json)
            biom_df['url'] = str(r.links.self)
            biomes_dfs.append(biom_df)
    if(biomes_dfs == []):
        return [None,None]
    main_biomes_df = pd.concat(biomes_dfs)

    # main_biomes_df = main_biomes_df.dropna()

    return [1,main_biomes_df]

df_samples = GetBiomSamplesByIds(biom)

df_samples

,type,id,attributes.longitude,attributes.sample-metadata,attributes.latitude,attributes.biosample,attributes.accession,attributes.collection-date,attributes.sample-desc,attributes.environment-biome,attributes.environment-feature,attributes.environment-material,attributes.sample-name,attributes.sample-alias,attributes.last-update,relationships.biome.data.id,relationships.biome.data.type,relationships.studies.data,url,attributes.analysis-completed
0,samples,SRS2329447,-151.0007,"[{'key': 'geographic location (longitude)', 'v...",-32.4957,SAMN07136875,SRS2329447,2011-06-21,Keywords: GSC:MIxS MIMS:5.0,ocean_biome,ocean,water,S0413,S0413,2023-09-21T16:35:08,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00005294', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,NaN
0,samples,ERS493132,-153.0658,"[{'key': 'temperature', 'value': '28.327', 'un...",6.6710,SAMEA2622957,ERS493132,2011-09-10,This sample (TARA_N000002310) was collected du...,marine biome (ENVO:00000447),surface water layer (ENVO:00002042),"particulate matter, including plankton (ENVO:x...",TARA_N000002310,TARA_N000002310,2023-09-21T16:27:38,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00001306', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2018-06-07
0,samples,SRS2329525,-68.8755,"[{'key': 'geographic location (longitude)', 'v...",38.3243,SAMN07136598,SRS2329525,2011-11-12,Keywords: GSC:MIxS MIMS:5.0,ocean_biome,ocean,water,S0122,S0122,2023-09-21T16:25:19,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00005294', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,NaN
0,samples,SRS2329438,5.5272,"[{'key': 'geographic location (longitude)', 'v...",-40.0627,SAMN07136934,SRS2329438,2010-10-27,Keywords: GSC:MIxS MIMS:5.0,ocean_biome,ocean,water,S0472,S0472,2023-09-21T16:11:00,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00005294', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,NaN
0,samples,ERS494107,-79.6493,"[{'key': 'temperature', 'value': '18.096139', ...",29.8556,SAMEA2623581,ERS494107,2012-01-17,This sample (TARA_N000003149) was collected du...,marine biome (ENVO:00000447),mesopelagic zone (ENVO:00000213),"particulate matter, including plankton (ENVO:x...",TARA_N000003149,TARA_N000003149,2023-09-21T16:09:41,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00001306', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2018-06-07
0,samples,ERS492023,-85.2732,"[{'key': 'temperature', 'value': '19.566255', ...",-5.2669,SAMEA2622230,ERS492023,2011-04-22,This sample (TARA_N000001652) was collected du...,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"particulate matter, including plankton (ENVO:x...",TARA_N000001652,TARA_N000001652,2023-09-21T16:04:20,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00001306', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2018-06-07
0,samples,ERS491719,-111.7202,"[{'key': 'temperature', 'value': '25.14875', '...",-25.8051,SAMEA2622010,ERS491719,2011-04-03,This sample (TARA_N000001574) was collected du...,marine biome (ENVO:00000447),surface water layer (ENVO:00002042),"particulate matter, including plankton (ENVO:x...",TARA_N000001574,TARA_N000001574,2023-09-21T16:01:06,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00001306', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,2018-06-07
0,samples,SRS2329521,-69.8629,"[{'key': 'geographic location (longitude)', 'v...",39.6923,SAMN07136594,SRS2329521,2011-11-08,Keywords: GSC:MIxS MIMS:5.0,ocean_biome,ocean,water,S0117,S0117,2023-09-21T15:53:38,root:Environmental:Aquatic:Marine,biomes,"[{'id': 'MGYS00005294', 'type': 'studies'}, {'...",https://www.ebi.ac.uk/metagenomics/api/v1/samp...,NaN
0,samples,SRS2329458,-64.1670,"[{'key': 'geographic location (longitude)', 'v...",31.6667,SAMN07136642,SRS2329458,2010-06-13,Keywor

In [4]:
sample_accessions = (df_samples['attributes.accession'].to_list())

studies = df_samples['relationships.studies.data'].to_list()

study_analaysises = []

studies_without_analysis = []

studies_Ids = []

for study in studies:
    print(study)
    # list of studies for sample
    s_ids_sample = []
    for s in str(study).split("'"):
        if('MGYS' in s):
            s_ids_sample.append(s) 
    # for each study
    for st in s_ids_sample:
        print(f'statt {st}')
        # test if we havent checked study
        if(st in studies_Ids):
            print(f'skipped {st} -duplicate- E')
            # we allready have an analysis
            break
        if(st not in studies_without_analysis):
            print(f'get analysis from {st}')
            study_analaysis = GetAnalysisFromStudie(st)
            if(study_analaysis[0] == None):
                # discard if studie has no analysis
                studies_without_analysis.append(st)
                print(f'{st} has no analysis')
                continue
            # keep if study has analysis
            studies_Ids.append(st)
            study_analaysises.append(study_analaysis[1])
            break
        else:
            print(f'skipped {st} -in s_w_a')
    #study_analaysises.append(study_analaysis)

study_analaysis = pd.concat(study_analaysises)

study_analaysis
"""


for study in studies_Ids:
    study_analaysis = GetAnalysisFromStudie(study)
    if(study_analaysis == None):
        continue
    study_analaysises.append(study_analaysis)


"""

[{'id': 'MGYS00005294', 'type': 'studies'}, {'id': 'MGYS00005780', 'type': 'studies'}]
statt MGYS00005294
get analysis from MGYS00005294


[{'id': 'MGYS00001306', 'type': 'studies'}, {'id': 'MGYS00002392', 'type': 'studies'}, {'id': 'MGYS00006491', 'type': 'studies'}]
statt MGYS00001306
get analysis from MGYS00001306
MGYS00001306 has no analysis
statt MGYS00002392
get analysis from MGYS00002392
[{'id': 'MGYS00005294', 'type': 'studies'}, {'id': 'MGYS00005780', 'type': 'studies'}]
statt MGYS00005294
skipped MGYS00005294 -duplicate- E
[{'id': 'MGYS00005294', 'type': 'studies'}, {'id': 'MGYS00005780', 'type': 'studies'}]
statt MGYS00005294
skipped MGYS00005294 -duplicate- E
[{'id': 'MGYS00001306', 'type': 'studies'}, {'id': 'MGYS00002392', 'type': 'studies'}, {'id': 'MGYS00006491', 'type': 'studies'}]
statt MGYS00001306
skipped MGYS00001306 -in s_w_a
statt MGYS00002392
skipped MGYS00002392 -duplicate- E
[{'id': 'MGYS00001306', 'type': 'studies'}, {'id': 'MGYS00002368', 'type': 'studies'}, {'id': 'MGYS00002392', 'type': 'studies'}, {'id': 'MGYS00006491', 'type': 'studies'}]
statt MGYS00001306
skipped MGYS00001306 -in s_w_a
st

'\n\n\nfor study in studies_Ids:\n    study_analaysis = GetAnalysisFromStudie(study)\n    if(study_analaysis == None):\n        continue\n    study_analaysises.append(study_analaysis)\n\n\n'

In [5]:
study_analaysis

,type,id,attributes.experiment-type,attributes.analysis-status,attributes.analysis-summary,attributes.pipeline-version,attributes.accession,attributes.is-private,attributes.complete-time,attributes.instrument-platform,attributes.instrument-model,relationships.run.data.id,relationships.run.data.type,relationships.study.data.id,relationships.study.data.type,relationships.sample.data.id,relationships.sample.data.type,url
0,analysis-jobs,MGYA00452399,metagenomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.1,MGYA00452399,False,2020-03-10T11:24:50,ILLUMINA,NextSeq 550,SRR5788454,runs,MGYS00005294,studies,SRS2329767,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-jobs,MGYA00452400,metagenomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.1,MGYA00452400,False,2020-02-03T13:07:00,ILLUMINA,NextSeq 550,SRR5788154,runs,MGYS00005294,studies,SRS2329586,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-jobs,MGYA00452401,metagenomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.1,MGYA00452401,False,2020-02-03T13:09:14,ILLUMINA,NextSeq 550,SRR5788004,runs,MGYS00005294,studies,SRS2329737,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-jobs,MGYA00452402,metagenomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.1,MGYA00452402,False,2020-02-03T13:11:14,ILLUMINA,NextSeq 550,SRR5788001,runs,MGYS00005294,studies,SRS2329739,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-jobs,MGYA00452403,metagenomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.1,MGYA00452403,False,2020-02-03T13:13:23,ILLUMINA,NextSeq 550,SRR5787999,runs,MGYS00005294,studies,SRS2329742,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,analysis-jobs,MGYA00163452,metatranscriptomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.0,MGYA00163452,False,2018-05-14T00:00:00,ILLUMINA,Illumina HiSeq 2000,ERR1712096,runs,MGYS00002368,studies,ERS490241,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-jobs,MGYA00163453,metatranscriptomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.0,MGYA00163453,False,2018-05-14T00:00:00,ILLUMINA,Illumina HiSeq 2000,ERR1711991,runs,MGYS00002368,studies,ERS488908,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-jobs,MGYA00163454,metatranscriptomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.0,MGYA00163454,False,2018-05-14T00:00:00,ILLUMINA,Illumina HiSeq 2000,ERR1712043,runs,MGYS00002368,studies,ERS488464,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-jobs,MGYA00163455,metatranscriptomic,completed,"[{'key': 'Submitted nucleotide sequences', 'va...",4.0,MGYA00163455,False,2018-05-14T00:00:00,ILLUMINA,Illumina HiSeq 2000,ERR1712021,runs,MGYS00002368,studies,ERS490941,samples,https://www.ebi.ac.uk/metagenomics/api/v1/anal...


In [6]:
analysis_accesions = []

for sample_acc in sample_accessions:
    acc = study_analaysis[study_analaysis['relationships.sample.data.id'] == sample_acc]['attributes.accession'].to_list()
    analysis_accesions.append([len(acc),acc])

analysis_accesions

[[1, ['MGYA00452486']],
 [2, ['MGYA00167300', 'MGYA00167342']],
 [1, ['MGYA00452585']],
 [1, ['MGYA00452480']],
 [2, ['MGYA00167038', 'MGYA00167501']],
 [1, ['MGYA00167937']],
 [1, ['MGYA00167302']],
 [1, ['MGYA00452594']],
 [1, ['MGYA00452522']],
 [1, ['MGYA00167024']]]

In [11]:

for analysis_sample in analysis_accesions:
    url = ""
    for analysis_id in analysis_sample[1]:
        print(f"start grab for {analysis_id}")
        #########################################
        # Get all downloads for one analysis
        #########################################

        with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
            
            dfs = []
            for r in mgnify.iterate(f'analyses/{analysis_id}/downloads'):
                df = pd.json_normalize(r.json)
                df['url'] = str(r.links.self)
                dfs.append(df)
            
        main_df = pd.concat(dfs)

        #print(main_df)

        ##########################
        # get specific data url
        #########################

        data_type = "TSV"

        #########################
        # get URL for data_label = "OTUs and taxonomic assignments for SSU rRNA"
        #########################

        data_label = "OTUs and taxonomic assignments for SSU rRNA"
        pipelin_version = 5.0
        c1 = main_df["attributes.file-format.name"] == data_type
        c2 = main_df["attributes.description.description"] == data_label
        c3 = main_df["relationships.pipeline.data.id"] == pipelin_version
        if(main_df.loc[(c1 & c2), "url"].size == 0):
            # if we dont get a match try the same with data_label = 'All reads encoding SSU rRNA'
            data_label = "All reads encoding SSU rRNA"
            c2 = main_df["attributes.description.description"] == data_label
            if(main_df.loc[(c1 & c2), "url"].size == 0):
                continue
            else:
                url = main_df.loc[(c1 & c2), "url"].iloc[0]
                break
        url = main_df.loc[(c1 & c2), "url"].iloc[0]
        break

    if(url == ""):
        print(f'no analysis found for {analysis_sample}')
        continue

    ##########################
    # download the data
    #########################

    data_output_folder = 'outputs/collection'
    os.makedirs(data_output_folder, exist_ok=True)

    response = requests.get(url)

    if not response:
        print(f"Could not download file, got response: {response.status_code}")
        break
        
    print(url)

    data_output_path = os.path.join(data_output_folder, f"{analysis_id}.tsv")
    with open(data_output_path, "w") as f:
        f.write(response.text)
        
    # add to galaxy (manually)
    #put(data_output_path)
     

start grab for MGYA00452486
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00452486/file/SRR5788294_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00167300
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00167300/file/ERR1756161_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00452585
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00452585/file/SRR5788216_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00452480
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00452480/file/SRR5788303_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00167038
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00167038/file/ERR1879702_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00167937
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00167937/file/ERR562553_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00167302
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00167302/file/ERR562500_MERGED_FASTQ_SSU_OTU.tsv
start grab for MGYA00452594
https://www.ebi.ac.uk/metagenomics/api/v1/a

In [8]:
from jsonapi_client import Session
import pandas as pd
import numpy as np
import requests
import os

# MGYA00678807
analysis_sample = ['MGYA00452486']


for analysis_id in analysis_sample:
    print(f"start grab for {analysis_id}")
        #########################################
        # Get all downloads for one analysis
        #########################################

    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
            
        dfs = []
        for r in mgnify.iterate(f'analyses/{analysis_id}/downloads'):
            df = pd.json_normalize(r.json)
            df['url'] = str(r.links.self)
            dfs.append(df)
            
    main_df = pd.concat(dfs)

    #print(main_df)

        ##########################
        # get specific data url
        #########################

    data_type = "TSV"
    data_label = "OTUs and taxonomic assignments for SSU rRNA"
    c1 = main_df["attributes.file-format.name"] == data_type
    c2 = main_df["attributes.description.description"] == data_label
    main_df = main_df.loc[(c1)]
main_df

start grab for MGYA00452486


,type,id,attributes.alias,attributes.file-format.name,attributes.file-format.extension,attributes.file-format.compression,attributes.description.label,attributes.description.description,attributes.group-type,attributes.file-checksum.checksum,attributes.file-checksum.checksum-algorithm,relationships.pipeline.data.id,relationships.pipeline.data.type,url
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_InterPro_1.tsv.gz,SRR5788294_MERGED_FASTQ_InterPro_1.tsv.gz,TSV,tsv,True,InterPro matches,InterPro matches (TSV),Functional analysis,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_InterPro_2.tsv.gz,SRR5788294_MERGED_FASTQ_InterPro_2.tsv.gz,TSV,tsv,True,InterPro matches,InterPro matches (TSV),Functional analysis,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_InterPro_3.tsv.gz,SRR5788294_MERGED_FASTQ_InterPro_3.tsv.gz,TSV,tsv,True,InterPro matches,InterPro matches (TSV),Functional analysis,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_SSU_MAPSeq.mseq.gz,SRR5788294_MERGED_FASTQ_SSU_MAPSeq.mseq.gz,TSV,tsv,True,MAPseq SSU assignments,MAPSeq output file for SSU,Taxonomic analysis SSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_SSU_OTU.tsv,SRR5788294_MERGED_FASTQ_SSU_OTU.tsv,TSV,tsv,False,Reads encoding SSU rRNA,All reads encoding SSU rRNA,Taxonomic analysis SSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_LSU_MAPSeq.mseq.gz,SRR5788294_MERGED_FASTQ_LSU_MAPSeq.mseq.gz,TSV,tsv,True,MAPseq LSU assignments,MAPSeq output file for LSU,Taxonomic analysis LSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
0,analysis-job-downloads,SRR5788294_MERGED_FASTQ_LSU_OTU.tsv,SRR5788294_MERGED_FASTQ_LSU_OTU.tsv,TSV,tsv,False,Reads encoding LSU rRNA,All reads encoding LSU rRNA,Taxonomic analysis LSU rRNA,,,4.1,pipelines,https://www.ebi.ac.uk/metagenomics/api/v1/anal...
